In [57]:
from os.path import join
from glob import glob
import pandas as pd

In [58]:
df_list = glob("/home/patel_zeel/kdd24_brick_kilns/data/pred_labeled/*.csv")
print(list(map(lambda x: x.split("/")[-1], df_list)))

df = pd.concat([pd.read_csv(name) for name in df_list])
df.shape

['muz.csv', 'dar.csv', 'od.csv', 'wb.csv', 'up.csv', 'mewat.csv', 'madhu.csv']


(4618, 8)

In [59]:
df.head(2)

,annotation_id,annotator,choice,created_at,id,image,lead_time,updated_at
0,2285,1,BK,2024-02-08T10:29:19.889691Z,9004,/data/local-files/?d=home/patel_zeel/bkdb/biha...,24.598,2024-02-08T11:05:30.106610Z
1,2286,1,BK,2024-02-08T10:30:41.231889Z,9005,/data/local-files/?d=home/patel_zeel/bkdb/biha...,65.845,2024-02-08T10:30:41.231905Z


In [60]:
# df.image.iloc[1000]

In [79]:
df["district"] = df["image"].apply(lambda x: x.split("/")[-2])
print(df.district.unique())
print(len(df.district.unique()))
df.head(2)

['Muzaffarpur' 'Darbhanga' 'munger' 'saharsa' 'madhepura' 'fazilka'
 'gurudaspur' 'harshipura' 'pathankot' 'REWARI' 'SIRSA' 'YAMUNANAGAR'
 'KARNAL' 'FATEHABAD' 'HISAR' 'KOLKATA' 'DAKSHIN%20DINAJPUR' 'HAMIRPUR'
 'CHITRAKOOT' 'KANNAUJ' 'KAUSHAMBI' 'JAUNPUR' 'mewat' 'madhubani']
24


,annotation_id,annotator,choice,created_at,id,image,lead_time,updated_at,district,lat,lon,lat_lag,lon_lag,real_lat,real_lon
0,2285,1,BK,2024-02-08T10:29:19.889691Z,9004,/data/local-files/?d=home/patel_zeel/bkdb/biha...,24.598,2024-02-08T11:05:30.106610Z,Muzaffarpur,25.91,85.55,-1,2,25.908,85.554
1,2286,1,BK,2024-02-08T10:30:41.231889Z,9005,/data/local-files/?d=home/patel_zeel/bkdb/biha...,65.845,2024-02-08T10:30:41.231905Z,Muzaffarpur,25.91,85.55,-2,2,25.906,85.554


In [69]:
df["lat"] = df["image"].apply(lambda x: float(x.split("/")[-1].split("%2C")[0]))
df["lon"] = df["image"].apply(lambda x: float(x.split("/")[-1].split("%2C")[1].split("_")[0]))
df["lat_lag"] = df["image"].apply(lambda x: int(x.split("/")[-1].split("%2C")[1].split("_")[1]))
df["lon_lag"] = df["image"].apply(lambda x: int(x.split("/")[-1].split("%2C")[1].split("_")[2].split(".")[0]))
df.head(2)

,annotation_id,annotator,choice,created_at,id,image,lead_time,updated_at,district,lat,lon,lat_lag,lon_lag
0,2285,1,BK,2024-02-08T10:29:19.889691Z,9004,/data/local-files/?d=home/patel_zeel/bkdb/biha...,24.598,2024-02-08T11:05:30.106610Z,Muzaffarpur,25.91,85.55,-1,2
1,2286,1,BK,2024-02-08T10:30:41.231889Z,9005,/data/local-files/?d=home/patel_zeel/bkdb/biha...,65.845,2024-02-08T10:30:41.231905Z,Muzaffarpur,25.91,85.55,-2,2


In [70]:
df.choice.value_counts().sum()

4578

In [71]:
7277+1628

8905

In [72]:
8905 - 4578

4327

In [73]:
df.drop_duplicates(subset=["image"]).shape

(4452, 13)

In [74]:
df.dtypes

annotation_id      int64
annotator          int64
choice            object
created_at        object
id                 int64
image             object
lead_time        float64
updated_at        object
district          object
lat              float64
lon              float64
lat_lag            int64
lon_lag            int64
dtype: object

In [76]:
df["real_lat"] = df["lat"] + 0.002 * df["lat_lag"]
df["real_lon"] = df["lon"] + 0.002 * df["lon_lag"]
df.head(2)

,annotation_id,annotator,choice,created_at,id,image,lead_time,updated_at,district,lat,lon,lat_lag,lon_lag,real_lat,real_lon
0,2285,1,BK,2024-02-08T10:29:19.889691Z,9004,/data/local-files/?d=home/patel_zeel/bkdb/biha...,24.598,2024-02-08T11:05:30.106610Z,Muzaffarpur,25.91,85.55,-1,2,25.908,85.554
1,2286,1,BK,2024-02-08T10:30:41.231889Z,9005,/data/local-files/?d=home/patel_zeel/bkdb/biha...,65.845,2024-02-08T10:30:41.231905Z,Muzaffarpur,25.91,85.55,-2,2,25.906,85.554


In [80]:
df.drop_duplicates(subset=["image"])[['real_lat', 'real_lon', 'district']].rename(columns={"real_lat": "lat", "real_lon": "lon"}).to_csv("temp.csv", index=False)